In [2]:
%run /eos/project/l/liu/Toolbox/myToolbox.py

%config InlineBackend.figure_format = 'retina'

plt.rcParams["mathtext.fontset"] = "cm"

from cpymad.madx import Madx

/eos/project/l/liu/Toolbox/myToolbox.py:24: FutureWarning: 'pandas.tseries.converter.register' has been moved and renamed to 'pandas.plotting.register_matplotlib_converters'. 
  converter.register()


Version 0.03. This is the latest version.
Please help me to improve it reporting bugs to guido.sterbini@cern.ch.
Your platform is Linux-3.10.0-957.21.3.el7.x86_64-x86_64-with-centos-7.6.1810-Core
Your folder is /eos/home-a/ahuschau/www/test-acc-models/repository/PS/2019/scenarios/SFTPRO/5_phase_rotation/notebooks
Your IP is 172.17.0.18
2019-08-19 22:26:30


# MTE phase rotation

In [3]:
# instances along the cycle at which the phase space will be evaluated
dt = 5
times = np.arange(800, 840, dt)
# number of turns to be tracked
turns = 2000
# flag to perform matching
match = 0
# flag to perform tracking
ptc_track = 1

MTE_table = pnd.read_pickle('./MTE_rotation_table.pkl')
    
phase_space = pnd.read_pickle('./MTE_rotation_phase_space.pkl')
# phase_space[phase_space.columns[1:8]] = np.nan
# phase_space[phase_space.columns[9:]] = np.nan

# x = ['X' + str(t) for t in times]
# px = ['PX' + str(t) for t in times]
# cols = x + px

# phase_space = pnd.DataFrame(columns = cols, index = np.arange(0, 40 * 2000, 1))

circuit_names = MTE_table.columns[:-5]

if 0:
    for t in times:
        # number of particles to be tracked
        n_p = 28
        # distance between phase space trajectories in mm and mrad
        dx = 1
        dpx = 0.11
    
        print('Cycle time ' + str(t) + ' ms')
        print('________________________________________________\n')

        # Setting up the MAD-X environment
        madx = Madx()
        madx.chdir('/eos/user/a/ahuschau/www/test-acc-models/repository/PS/2019/scenarios/SFTPRO/4_horizontal_splitting/')
        # madx = Madx(stdout=False)
        madx.command.beam('PARTICLE=PROTON, PC = 14.;')
        madx.input('BRHO := BEAM->PC * 3.3356;')
        madx.call('../../../PS_MU.seq')
        madx.call('../../../PS_SS.seq')
        madx.call('../../../PS.str')
        madx.call('../3_resonance_crossing/PS_RC_SFTPRO.str')
    #     madx.call('PS_HS_SFTPRO.madx')
        madx.call('../../../matching_macros.ptc')

        # defining the strenghts of the MTE elements
        for c in circuit_names:
            madx.input(c + ' = ' + str(MTE_table[c].loc[t]) + ';')

        # define LEQ settings
        for k in ['kd', 'kf']:
            madx.input(k + ' = ' + str(MTE_table[k].loc[t]) + ';')

        # redefining the horizontal tune
        Qx = MTE_table['Qx_input'].loc[t] - 6
        madx.input('Qx := ' + str(np.round(Qx, 6)) + ';' )
        madx.input('Qy := 0.29826;')

        if match:
            
            # update LEQ settings based on matching results of previous time step 
            for k in ['kd', 'kf']:
                madx.input(k + ' = ' + str(MTE_table[k].loc[t-dt]) + ';')
            
            print('\n________________________________________________')
            print('Matching of the horizontal tune with the LEQ...')
            print('________________________________________________\n')

            madx.command.use(sequence = 'PS')
            madx.command.match('use_macro;')
            madx.command.vary('name = kf, step = 1.0e-6;')
            madx.command.vary('name = kd, step = 1.0e-6;')
            madx.input('use_macro, name = ptc_twiss_tune_macro;')
            madx.input('constraint, expr = table(ptc_twiss_summary,Q1) = Qx;')
            madx.input('constraint, expr = table(ptc_twiss_summary,Q2) = Qy;')
            madx.input('jacobian, calls=50000, bisec=3, tolerance=1.0E-21;')
            madx.input('ENDMATCH;')

            # fill the MTE table with the matching results
            MTE_table['Qx'].loc[t] = madx.table['ptc_twiss_summary'].Q1
            MTE_table['Qy'].loc[t] = madx.table['ptc_twiss_summary'].Q2
            MTE_table['kd'].loc[t] = madx.globals['kd']    
            MTE_table['kf'].loc[t] = madx.globals['kf']

        if ptc_track:
            
            print('\n________________________________________________')
            print('Tracking with PTC...')
            print('________________________________________________\n')

            madx.command.use(sequence = 'PS')
            madx.command.ptc_create_universe()
            madx.command.ptc_create_layout('time=false, model=2, exact=true, method=6, nst=5;')

            if t == 800:
                dx = 1.5
                n_p = 40
                x = np.arange(dx, (n_p + 1) * dx, dx)
                px = x * 0.0
            else:
                x = np.arange(dx, (n_p + 1) * dx, dx)
                px = np.arange(dpx, (n_p + 1) * dpx, dpx)
            
            for i,j in enumerate(x):
                madx.input('PTC_START, x=' + str(j*1e-3) + ', px=' + str(px[i]*1e-3) + ', y=0.0, py=0.0, t=0.0, pt=0.0;')

            madx.command.ptc_track('icase=4, turns=' + str(turns) + ', element_by_element;')
            madx.command.ptc_track_end()
            madx.command.ptc_end()

            # identify all output tables of ptc_track 
            tables = [table for table in madx.table.keys() if 'obs' in table]

            # arange all particle coordinates in two vectors for X and PX
            X = np.empty(0)
            PX = np.empty(0)

            for i in np.arange(0, len(tables), 1):
                track = madx.table[tables[i]]
                X = np.append(X, track.x)
                PX = np.append(PX, track.px)

            try:
                phase_space['X' + str(t)].iloc[:len(X)] = X*1e3
                phase_space['PX' + str(t)].iloc[:len(X)] = PX*1e3
            except ValueError:
                phase_space['X' + str(t)] = X[:n_p * turns]*1e3
                phase_space['PX' + str(t)] = PX[:n_p * turns]*1e3
            phase_space.to_pickle('./MTE_rotation_phase_space.pkl')
    
    MTE_table.to_pickle('./MTE_rotation_table.pkl')

# Create Bokeh plot

In [4]:
from bokeh.plotting import figure, output_file, output_notebook, show, save, ColumnDataSource
from bokeh.models import Legend, LinearAxis, Range1d, CustomJS, Slider, Span
from bokeh.layouts import row, column, gridplot

In [5]:
t0 = str(times[0])
tend = str(times[-1])
data = phase_space
data_visible = data[['X' + t0, 'PX' + t0]]
data_visible.rename(columns={'X' + t0: 'X', 'PX' + t0: 'PX'}, inplace = True)

source_available = ColumnDataSource(data)
source_visible = ColumnDataSource(data_visible)
MTE_elements = pnd.read_pickle('../../4_horizontal_splitting/notebooks/Strengths_of_MTE_elements.pkl')

f = figure(plot_width=450, plot_height=400, x_axis_label='x [mm]', y_axis_label="x' [mrad]" , x_range = Range1d(-65, 65, bounds="auto"), y_range = Range1d(-4.5, 4.5, bounds="auto"), tools="box_zoom, pan, reset", active_drag = 'box_zoom')

f.axis.major_label_text_font = 'times'
f.axis.axis_label_text_font = 'times'
f.axis.axis_label_text_font_style = 'normal'
f.outline_line_color = 'black'

f1 = figure(plot_width=500, plot_height=400, x_axis_label='Cycle time [ms]', y_axis_label="Current [A]" , x_range = Range1d(700, 840, bounds="auto"), y_range = Range1d(-500, 600, bounds="auto"), tools="box_zoom, pan, reset", active_drag = 'box_zoom', toolbar_location="right")

f1.axis.major_label_text_font = 'times'
f1.axis.axis_label_text_font = 'times'
f1.axis.axis_label_text_font_style = 'normal'
f1.outline_line_color = 'black'

# Adding the second y axis to the plot.  
f1.extra_y_ranges = {"Qx": Range1d(start = 6.245, end = 6.3)}
f1.add_layout(LinearAxis(y_range_name="Qx", axis_label='Qx', axis_label_text_font = 'times', axis_label_text_font_style = 'normal', major_label_text_font = 'times'), 'right')
    
f.scatter('X', 'PX', source = source_visible, marker = "circle", size = 0.1, color = 'black')

col = ['black', 'gray', 'firebrick', 'royalblue', 'darkviolet', 'cadetblue']

legend_items = []

for i, circuit in enumerate(MTE_elements.columns[:6]):
    if circuit == 'Qx':
        c = f1.line(MTE_elements.index, MTE_elements[circuit], color = col[i], muted_color = col[i], muted_alpha = 0.2, y_range_name="Qx", line_dash = 'dashed')
        legend_items.append((circuit, [c]))
    else:
        c = f1.line(MTE_elements.index, MTE_elements[circuit], color = col[i], muted_color = col[i], muted_alpha = 0.2)
        legend_items.append((circuit, [c]))
    
time_slider = Slider(title = "Cycle time [ms]", start = int(t0), end = int(tend), value = int(t0), step=5)

vline = Span(location = time_slider.value, dimension = 'height', line_color = 'black', line_dash = 'dashed', line_width = 1)
f1.renderers.extend([vline])

time_slider.callback = CustomJS(
    args=dict(source_visible = source_visible,
              source_available = source_available,
              span = vline), 
        
        code =
    """
        var t = cb_obj.value;
        
        // Get the data from the data sources
        var data_visible = source_visible.data;
        var data_available = source_available.data;
        
        // Change y-axis data according to the selected value
        data_visible['X'] = data_available['X' + t.toString()];
        data_visible['PX'] = data_available['PX' + t.toString()];
        
        span.location = t;
        
        // Update the plot
        source_visible.change.emit();
    """)

layout = column(f, time_slider)
# grid = gridplot([[time_slider], [f], [f1]])
grid = gridplot([[f, f1], [time_slider], []])
legend = Legend(items = legend_items, location="bottom_center")
legend.orientation = "horizontal"
f1.add_layout(legend, 'below')
f1.legend.label_text_font_size = '9pt'
f1.legend.label_text_font = 'times'

legend.click_policy="mute"

# output_file('slider.html')

# output_notebook()
# show(grid)

output_file('../PS_PR_SFTPRO_2.html', mode="inline")
save(grid)

/cvmfs/sft.cern.ch/lcg/views/LCG_96/x86_64-centos7-gcc8-opt/lib/python2.7/site-packages/pandas/core/frame.py:4025: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


'/eos/home-a/ahuschau/www/test-acc-models/repository/PS/2019/scenarios/SFTPRO/5_phase_rotation/PS_PR_SFTPRO_2.html'